In [ ]:
%config Completer.use_jedi = False

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import lightgbm
from IPython.display import display
from sklearn.model_selection import train_test_split, GroupKFold, KFold
from sklearn.metrics import mean_absolute_error
import optuna
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import normalize

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

# Basic stats and head of the data

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train = train.mask(train<0,0)

In [ ]:
train.describe()

In [ ]:
sns.heatmap(train.corr(), annot=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_input = MinMaxScaler()

In [ ]:
print(train.head())

In [ ]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)

train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

train = scaler_input.fit_transform(train.to_numpy()).reshape(75450, 80, 5)
test = scaler_input.transform(test.to_numpy())

from sklearn.model_selection import train_test_split
X_train,X_valid,Y_train,Y_valid = train_test_split(train,targets, train_size = 0.75, shuffle=True)

In [ ]:
print("X_train : {}\nX_valid : {}\nY_train : {}\nY_valid : {}".format(X_train.shape,X_valid.shape,Y_train.shape,Y_valid.shape))

# Solution with LSTM

In [ ]:
# train['u_in_cumsum'] = (train['u_in']).groupby(train['breath_id']).cumsum()
# test['u_in_cumsum'] = (test['u_in']).groupby(test['breath_id']).cumsum()

In [ ]:
# train['u_in_cumsum'].iloc[:500].plot()
# train['u_in'].iloc[:500].plot()

In [ ]:
# train['u_in'].iloc[:500].plot()

In [ ]:
# train['u_in_lag'] = train['u_in'].shift(2)
# train.info()
# train = train.fillna(0)

# test['u_in_lag'] = test['u_in'].shift(2)
# test = test.fillna(0)

In [ ]:
import matplotlib.pyplot as plt
def plt_model_loss(build_model):# list all data in history
    print(build_model.history.keys())
    # summarize history for accuracy
    plt.plot(build_model.history['loss'])
    plt.plot(build_model.history['val_loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train,X_valid,Y_train,Y_valid = train_test_split(train,targets,train_size = 0.75,random_state = 20)

In [ ]:
scheduler = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 200*((len(train)*0.8)/1024), 1e-5)
    
model = keras.models.Sequential([
    keras.layers.Input(shape=(80, 5)),
    keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(1),
])

model.compile(optimizer="adam", loss="mae")

In [ ]:
print(model.summary())

In [ ]:
build_model = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), epochs=100, batch_size=120, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler)])

In [ ]:
plt_model_loss(build_model)

In [ ]:
test_pred = model.predict(test.reshape(50300, 80, 5)).squeeze().reshape(-1, 1).squeeze()
submission["pressure"] = sum(test_pred)
submission.to_csv('submission.csv', index=False)

In [ ]:
import pandas as pd
pd.read_csv("submission.csv")

In [ ]:
# kf = KFold(n_splits=2, shuffle=True, random_state=1)
# test_preds = []
# for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
#     print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
#     X_train, X_valid = train[train_idx], train[test_idx]
#     y_train, y_valid = targets[train_idx], targets[test_idx]
    
#     scheduler = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 200*((len(train)*0.8)/1024), 1e-5)
    
#     model = keras.models.Sequential([
#         keras.layers.Input(shape=(80, 6)),
#         keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True, dropout = 0.2)),
#         keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True,dropout = 0.2)),
#         keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True,dropout = 0.2)),
#         keras.layers.Dense(50, activation='relu'),
#         keras.layers.Dense(25, activation='relu'),
#         keras.layers.Dense(1),
#     ])
#     model.compile(optimizer="adam", loss="mae")
#     build_model = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100, batch_size=1024, callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler)])
#     plt_model_loss(build_model)
#     model.save(f'Fold{fold+1} RNN Weights')
#     test_preds.append(model.predict(test.to_numpy().reshape(50300, 80, 6)).squeeze().reshape(-1, 1).squeeze())

In [ ]:
# submission["pressure"] = sum(test_preds)/4
# submission.to_csv('submission.csv', index=False)